## Cool stuff from Analytics for Humans using the GA reporting API
##### Article url --> https://medium.com/analytics-for-humans/submitting-your-first-google-analytics-reporting-api-request-cdda19969940

In [1]:
#Packages needed for authentication
import httplib2 as lib2 #Example of the "as" function
import google.oauth2.credentials #Importing a sub-package
from google_auth_httplib2 import AuthorizedHttp


#Packages needed for connecting with Google API
from googleapiclient.discovery import build as google_build #An example with all the statements together

#Data processing packages
import pandas
import numpy
import json
from datetime import datetime, timedelta #importing multiple sub-packages from one package


In [2]:
#We have this from previous sections
access_token = ""
refresh_token = ""
client_id = ""
client_secret = ""
    

#This is consistent for all Google services
token_uri = 'https://accounts.google.com/o/oauth2/token'

#We are essentially setting the expiry date to 1 day before today, which will make it always expire
token_expiry = datetime.now() - timedelta(days = 1)

#¯\_(ツ)_/¯
user_agent = 'my-user-agent/1.0'

#The real code that initalized the client
credentials = google.oauth2.credentials.Credentials(access_token,  
                                                    refresh_token=refresh_token,
                                                    token_uri='https://accounts.google.com/o/oauth2/token',
                                                    client_id=client_id,
                                                    client_secret=client_secret)

#Authorize client
authorized = AuthorizedHttp(credentials=credentials)

In [3]:
#API Name and Verison, these don't change until 
#they release a new API version for us to play with. 
api_name = 'analyticsreporting'
api_version = 'v4'

#Let's build the client
api_client = google_build(serviceName=api_name, version=api_version, http=authorized)

In [7]:
sample_request = {
      'viewId': '116673378',
      'dateRanges': {
          'startDate': datetime.strftime(datetime.now() - timedelta(days = 30),'%Y-%m-%d'),
          'endDate': datetime.strftime(datetime.now(),'%Y-%m-%d')
      },
      'dimensions': [{'name': 'ga:date'}],
      'metrics': [{'expression': 'ga:sessions'}]
    }

response = api_client.reports().batchGet(
      body={
        'reportRequests': sample_request
      }).execute()
print(response)

{'reports': [{'columnHeader': {'dimensions': ['ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}, 'data': {'rows': [{'dimensions': ['20200211'], 'metrics': [{'values': ['214']}]}, {'dimensions': ['20200212'], 'metrics': [{'values': ['197']}]}, {'dimensions': ['20200213'], 'metrics': [{'values': ['398']}]}, {'dimensions': ['20200214'], 'metrics': [{'values': ['266']}]}, {'dimensions': ['20200215'], 'metrics': [{'values': ['275']}]}, {'dimensions': ['20200216'], 'metrics': [{'values': ['180']}]}, {'dimensions': ['20200217'], 'metrics': [{'values': ['185']}]}, {'dimensions': ['20200218'], 'metrics': [{'values': ['227']}]}, {'dimensions': ['20200219'], 'metrics': [{'values': ['205']}]}, {'dimensions': ['20200220'], 'metrics': [{'values': ['185']}]}, {'dimensions': ['20200221'], 'metrics': [{'values': ['151']}]}, {'dimensions': ['20200222'], 'metrics': [{'values': ['137']}]}, {'dimensions': ['20200223'], 'metrics': [{'values': ['172']}]}, {'di

In [5]:
#Parse the response of API
def prase_response(report):

    """Parses and prints the Analytics Reporting API V4 response"""
    #Initialize results, in list format because two dataframes might return
    result_list = []

    #Initialize empty data container for the two dateranges (if there are two that is)
    data_csv = []
    data_csv2 = []

    #Initialize header rows
    header_row = []

    #Get column headers, metric headers, and dimension headers.
    columnHeader = report.get('columnHeader', {})
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    dimensionHeaders = columnHeader.get('dimensions', [])

    #Combine all of those headers into the header_row, which is in a list format
    for dheader in dimensionHeaders:
        header_row.append(dheader)
    for mheader in metricHeaders:
        header_row.append(mheader['name'])

    #Get data from each of the rows, and append them into a list
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        row_temp = []
        dimensions = row.get('dimensions', [])
        metrics = row.get('metrics', [])
        for d in dimensions:
            row_temp.append(d)
        for m in metrics[0]['values']:
            row_temp.append(m)
        data_csv.append(row_temp)

        #In case of a second date range, do the same thing for the second request
        if len(metrics) == 2:
            row_temp2 = []
            for d in dimensions:
                row_temp2.append(d)
            for m in metrics[1]['values']:
                row_temp2.append(m)
            data_csv2.append(row_temp2)

    #Putting those list formats into pandas dataframe, and append them into the final result
    result_df = pandas.DataFrame(data_csv, columns=header_row)
    result_list.append(result_df)
    if data_csv2 != []:
        result_list.append(pandas.DataFrame(data_csv2, columns=header_row))

    return result_list

In [8]:
response_data = response.get('reports', [])[0]
print(prase_response(response_data)[0])

     ga:date ga:sessions
0   20200211         214
1   20200212         197
2   20200213         398
3   20200214         266
4   20200215         275
5   20200216         180
6   20200217         185
7   20200218         227
8   20200219         205
9   20200220         185
10  20200221         151
11  20200222         137
12  20200223         172
13  20200224         153
14  20200225         179
15  20200226         196
16  20200227         143
17  20200228         194
18  20200229         156
19  20200301         145
20  20200302         136
21  20200303         260
22  20200304         319
23  20200305         252
24  20200306         260
25  20200307         206
26  20200308         173
27  20200309         229
28  20200310         221
29  20200311         210
30  20200312         133
